In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:1


In [2]:
def true_1D_1(x): #True function for 1D_1 dy2/dx2 + dy/dx - 6y = 0; BC1: y(0)=2; BC2: dy/dx at (x=0) = -1;
    y = np.exp(2*x) + np.exp(-3*x)
    return y
    

In [3]:
loss_thresh = 0.005
label = "1D_SODE_rowdy"

x = np.linspace(0,2,100).reshape(-1,1)

bc1_x = x[0].reshape(-1,1)
bc1_y = true_1D_1(x[0]).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)
    

bc2_x = x[0].reshape(-1,1)
x_bc2_train = torch.from_numpy(bc2_x).float().to(device)
bc2_val = torch.tensor(-1.0,device=device)
bc2_val = bc2_val.view(1,1)

x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)
y_true = true_1D_1(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

In [4]:
def colloc_pts(N_f,seed):
    #Collocation Points
    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,y)
    x01 = np.array([[0.0, 1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    
    x_coll_train = lb + (ub-lb)*sampling(N_f)
    x_coll_train = np.vstack((x_coll_train, bc1_x.reshape(-1,1))) # append training points to collocation points 

    return x_coll_train

In [5]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val,rowdy_terms):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
        
        
        self.omega1 = Parameter(torch.ones((len(layers)-2,1))) 
        
        
        self.alpha = Parameter(torch.zeros(rowdy_terms,len(layers)-2))
        self.omega = Parameter((1/n_val)*torch.ones(rowdy_terms,len(layers)-2))
        
        self.n = torch.tensor(n_val)
    
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.omega1[i,0]*z)
            for j in range(rowdy_terms):
                a = a + self.alpha[j,i]*self.n*torch.sin((j+1)*self.n*self.omega[j,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_BC2(self,x_bc2,bc2_val):
        g = x_bc2.clone()             
        g.requires_grad = True
        y = self.forward(g)    
            
        y_x = autograd.grad(y,g,torch.ones([x_bc2.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        dy_dx = y_x[:,[0]]
        
        bc2 = dy_dx
        
        loss_bc2= self.loss_function(bc2,bc2_val)

        return loss_bc2
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        y_xx = autograd.grad(y_x,g,torch.ones(x_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        dy2_d2x = y_xx[:,[0]]
        
        f = dy2_d2x + dy_dx - 6*y
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_bc2,bc2_val,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_bc2 = self.loss_BC2(x_bc2,bc2_val)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + loss_bc2 + loss_f
        
        return loss_val
          
    'test neural network'
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re

In [6]:
def train_step(x_coll,f_hat):
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [7]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    omega_val.append(PINN.omega.cpu().detach().numpy())
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [8]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*123)
    start_time = time.time()
    thresh_flag = 0

    for i in range(max_iter):
        x_coll = torch.from_numpy(colloc_pts(N_f,i*11)).float().to(device)
        f_hat = torch.zeros(x_coll.shape[0],1).to(device)
        train_step(x_coll,f_hat)
        
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_bc2_train,bc2_val,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [9]:
max_reps = 10
max_iter = 100


N_f = 1000

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []
omega_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val = 5.0
rowdy_terms = 2

for reps in range(max_reps):
    print(reps)

    train_loss = []
    test_mse_loss = []
    test_re_loss =[]
    alpha_val = []
    omega_val = []

    'Generate Training data'
    torch.manual_seed(reps*36)
     #Total number of collocation points 



    layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    PINN = Sequentialmodel(layers,n_val,rowdy_terms)
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25, 
                              max_iter = 10, 
                              max_eval = 15, 
                              tolerance_grad = 1e-5, 
                              tolerance_change = 1e-5, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    train_model(max_iter,reps)


    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    alpha_full.append(alpha_val)
    omega_full.append(omega_val)  

    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full, "test_re_loss": test_re_full, "Time": elapsed_time, "alpha": alpha_full, "omega": omega_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 4.3443027 Test MSE 386.51022543182546 Test RE 1.002173873411708
1 Train Loss 2.606071 Test MSE 382.9875848378914 Test RE 0.997596531397191
2 Train Loss 2.397064 Test MSE 383.704570772472 Test RE 0.9985298883312785
3 Train Loss 2.3842182 Test MSE 383.97660525184676 Test RE 0.998883788767

96 Train Loss 0.6299419 Test MSE 95.83965249626657 Test RE 0.4990398164790996
97 Train Loss 0.6262821 Test MSE 94.75974162551083 Test RE 0.4962202880201349
98 Train Loss 0.61982447 Test MSE 95.12998026155407 Test RE 0.4971887415784638
99 Train Loss 0.61541224 Test MSE 94.72437972118604 Test RE 0.4961276910406399
Training time: 71.86
Training time: 71.86
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Li

92 Train Loss 0.76271737 Test MSE 116.26969971904617 Test RE 0.5496621825077997
93 Train Loss 0.75797546 Test MSE 115.35355302937425 Test RE 0.5474923691794491
94 Train Loss 0.7527384 Test MSE 112.89664226972558 Test RE 0.5416304789932735
95 Train Loss 0.7197054 Test MSE 108.96684434708496 Test RE 0.5321202292798511
96 Train Loss 0.71320635 Test MSE 107.21386239355725 Test RE 0.5278226872440558
97 Train Loss 0.6984404 Test MSE 105.70704075704329 Test RE 0.5241004589899563
98 Train Loss 0.69488394 Test MSE 106.44568808417412 Test RE 0.5259283953105578
99 Train Loss 0.6882765 Test MSE 105.72541016683128 Test RE 0.524145995208801
Training time: 67.26
Training time: 67.26
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, b

88 Train Loss 0.43163052 Test MSE 63.208817330550545 Test RE 0.4052764934059438
89 Train Loss 0.42912462 Test MSE 63.28816656903959 Test RE 0.40553079570350736
90 Train Loss 0.41980347 Test MSE 61.92024807919887 Test RE 0.4011242576379726
91 Train Loss 0.41091177 Test MSE 60.084840398506934 Test RE 0.3951345800724092
92 Train Loss 0.40019822 Test MSE 60.145999271356985 Test RE 0.3953356277827722
93 Train Loss 0.38437173 Test MSE 56.72758589483219 Test RE 0.3839367935605864
94 Train Loss 0.36896503 Test MSE 55.21668576492888 Test RE 0.37878934167795764
95 Train Loss 0.3656722 Test MSE 54.72849328189296 Test RE 0.3771111110309978
96 Train Loss 0.3503491 Test MSE 51.366378103836524 Test RE 0.3653440633295721
97 Train Loss 0.33371413 Test MSE 48.638449663007684 Test RE 0.3555105099253795
98 Train Loss 0.32987252 Test MSE 48.30443990432066 Test RE 0.3542877268907029
99 Train Loss 0.32890576 Test MSE 48.032539419595714 Test RE 0.3532891960419192
Training time: 72.40
Training time: 72.40
Sequ

83 Train Loss 0.0070774467 Test MSE 0.051880645637080836 Test RE 0.011610883528620879
84 Train Loss 0.0066052023 Test MSE 0.025491316642505084 Test RE 0.008138768889921712
85 Train Loss 0.0064064744 Test MSE 0.011911965102576829 Test RE 0.005563579378631222
86 Train Loss 0.006085333 Test MSE 0.013215231947910895 Test RE 0.005860031864846258
87 Train Loss 0.005727466 Test MSE 0.026527902523946524 Test RE 0.00830259854587768
88 Train Loss 0.005550961 Test MSE 0.026825667445441365 Test RE 0.008349065164232249
89 Train Loss 0.0055436534 Test MSE 0.02689223418700306 Test RE 0.008359417669187243
90 Train Loss 0.005539143 Test MSE 0.027010615599824307 Test RE 0.008377796822564619
91 Train Loss 0.005532927 Test MSE 0.027283842581331416 Test RE 0.008420063177132033
92 Train Loss 0.00552989 Test MSE 0.02751936545270284 Test RE 0.008456327425785852
93 Train Loss 0.0055264193 Test MSE 0.027899964684839774 Test RE 0.008514603114775018
94 Train Loss 0.0055249454 Test MSE 0.028431924859207584 Test RE

...

72 Train Loss 0.078963995 Test MSE 8.84708779024765 Test RE 0.15162218501897
73 Train Loss 0.0750621 Test MSE 8.14869373443177 Test RE 0.14551460347191414
74 Train Loss 0.07409494 Test MSE 8.078881516205442 Test RE 0.14488992978704482
75 Train Loss 0.07175862 Test MSE 8.701342185571812 Test RE 0.15036809802948387
76 Train Loss 0.068701334 Test MSE 8.517458345761913 Test RE 0.14877076345306597
77 Train Loss 0.06574774 Test MSE 7.599160296639768 Test RE 0.14052234395250696
78 Train Loss 0.06341919 Test MSE 7.701580084319071 Test RE 0.14146613893730767
79 Train Loss 0.06185522 Test MSE 7.677725087160151 Test RE 0.14124687928195556
80 Train Loss 0.05716316 Test MSE 6.428139285564819 Test RE 0.12924246178913942
81 Train Loss 0.05011646 Test MSE 5.28489737017486 Test RE 0.1171873845388925
82 Train Loss 0.04863572 Test MSE 5.115783147015538 Test RE 0.11529716958063571
83 Train Loss 0.04370399 Test MSE 4.373943455135161 Test RE 0.10661029981370475
84 Train Loss 0.03839846 Test MSE 4.2307625669

67 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
68 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
69 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
70 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
71 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
72 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
73 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
74 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
75 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
76 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
77 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
78 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.9980756896952772
79 Train Loss 4.891897 Test MSE 383.35558080631847 Test RE 0.998

63 Train Loss 0.9460063 Test MSE 148.1954582824688 Test RE 0.620554643316958
64 Train Loss 0.9329438 Test MSE 143.32255155128811 Test RE 0.6102669464643733
65 Train Loss 0.8874643 Test MSE 137.02729277518586 Test RE 0.596713854342702
66 Train Loss 0.8579332 Test MSE 134.68563178115213 Test RE 0.5915932586080369
67 Train Loss 0.8408239 Test MSE 131.82845208647083 Test RE 0.5852846832222746
68 Train Loss 0.82544124 Test MSE 127.72529123030009 Test RE 0.5761041893018751
69 Train Loss 0.802184 Test MSE 123.52344732460257 Test RE 0.5665487482119704
70 Train Loss 0.77420694 Test MSE 116.97569789092587 Test RE 0.5513284515172053
71 Train Loss 0.71854913 Test MSE 109.64064863113303 Test RE 0.5337628955760001
72 Train Loss 0.7067124 Test MSE 106.04811764624637 Test RE 0.5249453156258186
73 Train Loss 0.6708326 Test MSE 101.56073738175043 Test RE 0.51371885420263
74 Train Loss 0.6578477 Test MSE 101.70641062348511 Test RE 0.5140871474876848
75 Train Loss 0.65173626 Test MSE 101.32549178386692 Te

59 Train Loss 0.8942568 Test MSE 136.65719051337405 Test RE 0.5959074658244912
60 Train Loss 0.8710714 Test MSE 131.26566266440895 Test RE 0.5840340262963751
61 Train Loss 0.8552012 Test MSE 129.21068190338937 Test RE 0.5794444290140257
62 Train Loss 0.8331138 Test MSE 122.64728600931993 Test RE 0.5645358855830847
63 Train Loss 0.7670526 Test MSE 114.58516999896887 Test RE 0.5456658682467684
64 Train Loss 0.74540496 Test MSE 111.73626858741052 Test RE 0.5388397978929981
65 Train Loss 0.72681165 Test MSE 108.04785169531456 Test RE 0.5298716095104564
66 Train Loss 0.6880718 Test MSE 101.69575663467482 Test RE 0.5140602208557989
67 Train Loss 0.641101 Test MSE 96.07095604704806 Test RE 0.4996416556797945
68 Train Loss 0.6310514 Test MSE 93.77629949614798 Test RE 0.4936386183122288
69 Train Loss 0.6138974 Test MSE 91.87452878775531 Test RE 0.4886075178450635
70 Train Loss 0.5900309 Test MSE 89.64029546057229 Test RE 0.48262989808324575
71 Train Loss 0.5811281 Test MSE 87.92341095805713 Tes

55 Train Loss 0.5283619 Test MSE 78.38638924950683 Test RE 0.45131826760458504
56 Train Loss 0.5233457 Test MSE 78.28571853219982 Test RE 0.4510283631243374
57 Train Loss 0.4917271 Test MSE 72.79066200715567 Test RE 0.43491102493563294
58 Train Loss 0.45171708 Test MSE 66.69782486904882 Test RE 0.41631150982694065
59 Train Loss 0.4225987 Test MSE 59.03042614171138 Test RE 0.3916521741347705
60 Train Loss 0.3867645 Test MSE 55.85105954032241 Test RE 0.38095904622458016
61 Train Loss 0.37220165 Test MSE 53.17504530548977 Test RE 0.3717205035789415
62 Train Loss 0.34980065 Test MSE 46.91277208102604 Test RE 0.34914685196464296
63 Train Loss 0.29532388 Test MSE 40.61727178858897 Test RE 0.32487624516216285
64 Train Loss 0.28907293 Test MSE 40.91478007146245 Test RE 0.32606388069524306
65 Train Loss 0.28216413 Test MSE 41.13267140060421 Test RE 0.32693095310622244
66 Train Loss 0.26944044 Test MSE 39.24546983574917 Test RE 0.3193429617777276
67 Train Loss 0.26527268 Test MSE 37.888685406812

In [10]:
lrnr_tune[tune_reps,2]

NameError: name 'lrnr_tune' is not defined

In [ ]:
import scipy.io as sio

In [ ]:
for tune_reps in range(75):
    label = "1D_SODE_rowdy_tune"+str(tune_reps)+".mat" #WRONGLY SAVED AS STAN - DOESN'T MATTER
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

In [11]:
a = 0
for i in range(10):
    a = a + test_re_full[i][-1]
print(a/10)

0.33411856570698817
